## RMMS to df

In [2]:
# pip install lxml beautifulsoup4 

In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import lxml
from bs4 import BeautifulSoup
import requests

from IPython.display import display

In [116]:
## FIX THIS ONE
file_path = 'Assessment1/testfile/RMMS/RMMS/N1.traffic.htm'

### First try for only the N1

In [119]:
# Step 1: Read the HTML table without skipping any rows initially
tables = pd.read_html(file_path, flavor='lxml')

# Step 2: Use the correct table index (based on your inspection, it's likely index 0)
raw_df = tables[2]

# Step 3: Drop rows that are entirely NaN
raw_df = raw_df.dropna(how='all')

# Step 4: Reset index to make processing easier
raw_df = raw_df.reset_index(drop=True)
raw_df = raw_df.drop([0, 1, 2])

# Step 5: Locate the actual column headers (usually at row index 1 or 2)
# For this file, headers appear to be on row index 1
row1 = raw_df.iloc[0].astype(str)
row2 = raw_df.iloc[1].astype(str)
combined_headers = row1 + "-" + row2
combined_headers = combined_headers.str.replace('-nan', '')

# Step 6: Create final DataFrame
df = raw_df[1:]  # Data starts after header row
df.columns = combined_headers  # Set 

# Step 7: Reset index
df = df.reset_index(drop=True)
df = df[df['Link no'].notna()]

df.rename(columns={df.columns[0]: "Link no"}, inplace=True)
df.set_index('Link no', inplace=True)

# Step 8: Drop rows that are entirely NaN except the first column
df = df.dropna(axis=0, how='all')

display(df)

,Name,Start location-LRP,Start location-Offset,Start location-Chainage,End location-LRP,End location-Offset,End location-Chainage,Length-(Km),Traffic Data-Heavy Truck,Traffic Data-Medium Truck,...,Traffic Data-Car,Traffic Data-Auto Rickshaw,Traffic Data-Motor Cycle,Traffic Data-Bi-Cycle,Traffic Data-Cycle Rickshaw,Traffic Data-Cart,Total-Motorized,Total-Non Motorized,Total-Total AADT,Traffic-(AADT)
Link no,,,,,,,,,,,,,,,,,,,,,
N1-1L,Jatrabari - Int.with Z1101 (Left) (Left),LRPS,0,0,LRPS,822,0.822,0.822,402.0,5842.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
N1-1R,Jatrabari - Int.with Z1101 (Left) (Right),LRPS,0,0,LRPS,822,0.822,0.822,660.0,6155.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
N1-2L,Int.with Z1101 - Signboard (Left) R111 (Left),LRPS,822,0.822,LRPS,4175,4.175,3.353,660.0,6155.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right),LRPS,822,0.822,LRPS,4175,4.175,3.353,402.0,5842.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
N1-3L,Signboard - Shimrail (Left)R110 (Left),LRPS,4175,4.175,LRPS,7181,7.181,3.006,91.0,2706.0,...,1690.0,2266.0,1087.0,75.0,1198.0,0.0,16288.0,1273.0,17561.0,17561.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
N1-65,Coxsbazar Link Road N110-Maricha Z1009,LRP386,724,381.481,LRP403,217,398.205,16.724,24.0,455.0,...,142.0,2476.0,419.0,97.0,1042.0,0.0,4681.0,1139.0,5820.0,5820.0
N1-66,Maricha Z1009-Ukhia Dakbanglo Z1503,LRP403,217,398.205,LRP413,300,408.434,10.229,24.0,455.0,...,142.0,2476.0,419.0,97.0,1042.0,0.0,4681.0,1139.0,5820.0,5820.0
N1-67,Ukhia Dakbanglo Z1503 - Gundum Z1504,LRP413,300,408.434,LRP420,900,416.044,7.610,24.0,455.0,...,142.0,2476.0,419.0,97.0,1042.0,0.0,4681.0,1139.0,5820.0,5820.0


## For all roads

In [123]:
directory_path = 'Assessment1/testfile/RMMS/RMMS/'

all_data = []

# Step 1: Loop through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('traffic.htm'):  # Only consider files that end with 'traffic.htm'
        # Construct full file path
        file_path = os.path.join(directory_path, filename)

        # Step 1: Read the HTML table without skipping any rows initially
        tables = pd.read_html(file_path, flavor='lxml')

        # Step 2: Use the correct table index (based on your inspection, it's likely index 0)
        raw_df = tables[2]

        # Step 3: Drop rows that are entirely NaN
        raw_df = raw_df.dropna(how='all')

        # Step 4: Reset index to make processing easier
        raw_df = raw_df.reset_index(drop=True)
        raw_df = raw_df.drop([0, 1, 2])

        # Step 5: Locate the actual column headers (usually at row index 1 or 2)
        # For this file, headers appear to be on row index 1
        row1 = raw_df.iloc[0].astype(str)
        row2 = raw_df.iloc[1].astype(str)
        combined_headers = row1 + "-" + row2
        combined_headers = combined_headers.str.replace('-nan', '')

        # Step 6: Create final DataFrame
        df = raw_df[1:]  # Data starts after header row
        df.columns = combined_headers  # Set 

        # Step 7: Reset index
        df = df.reset_index(drop=True)
        df = df[df['Link no'].notna()]

        df.rename(columns={df.columns[0]: "Link no"}, inplace=True)
        df.set_index('Link no', inplace=True)

        # Step 8: Drop rows that are entirely NaN except the first column
        df = df.dropna(axis=0, how='all')

        all_data.append(df)

# Step 11: Combine all DataFrames in the list into a single large DataFrame
final_df = pd.concat(all_data, axis=0)

# Optionally, display the final combined DataFrame
display(final_df)



,Name,Start location-LRP,Start location-Offset,Start location-Chainage,End location-LRP,End location-Offset,End location-Chainage,Length-(Km),Traffic Data-Heavy Truck,Traffic Data-Medium Truck,...,Traffic Data-Car,Traffic Data-Auto Rickshaw,Traffic Data-Motor Cycle,Traffic Data-Bi-Cycle,Traffic Data-Cycle Rickshaw,Traffic Data-Cart,Total-Motorized,Total-Non Motorized,Total-Total AADT,Traffic-(AADT)
Link no,,,,,,,,,,,,,,,,,,,,,
N1-1L,Jatrabari - Int.with Z1101 (Left) (Left),LRPS,0,0,LRPS,822,0.822,0.822,402.0,5842.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
N1-1R,Jatrabari - Int.with Z1101 (Left) (Right),LRPS,0,0,LRPS,822,0.822,0.822,660.0,6155.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
N1-2L,Int.with Z1101 - Signboard (Left) R111 (Left),LRPS,822,0.822,LRPS,4175,4.175,3.353,660.0,6155.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
N1-2R,Int.with Z1101 - Signboard (Left) R111 (Right),LRPS,822,0.822,LRPS,4175,4.175,3.353,402.0,5842.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
N1-3L,Signboard - Shimrail (Left)R110 (Left),LRPS,4175,4.175,LRPS,7181,7.181,3.006,91.0,2706.0,...,1690.0,2266.0,1087.0,75.0,1198.0,0.0,16288.0,1273.0,17561.0,17561.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z8915-3,Int.with Z8916 - Daulatkhan,LRP010,1150,10.47,LRP014,2290,15.66,5.190,0.0,77.0,...,25.0,1025.0,872.0,396.0,380.0,0.0,2341.0,776.0,3117.0,3117.0
Z8916-1,Guingerhat (Int.with R890) - Int.with Z8915,LRPS,0,0,LRP004,3626,7.626,7.626,0.0,0.0,...,15.0,672.0,640.0,453.0,524.0,0.0,1463.0,977.0,2440.0,2440.0
Z8916-2,Int.with Z8915 - Daulatkhan Bazar,LRP004,3626,7.626,LRP005,6600,11.591,3.965,0.0,0.0,...,15.0,672.0,640.0,453.0,524.0,0.0,1463.0,977.0,2440.0,2440.0


# Link this to the geometry datasets